# ## Crypto research and trading notebook using Grok search and Kraken API

## Install needed libs

In [22]:
# !pip install langchain
# !pip install langraph
# !pip install -U langchain-google-genai
# !pip install python-dotenv
# !pip install langchain-community
# !pip install playwright
# !pip install beautifulsoup4
# !pip install requests
# !pip install duckduckgo-search
# %pip install xai-sdk

## Import needed libs

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
# from langgraph.graph import StateGraph, END
# from langgraph.prebuilt import ToolNode
# from langgraph.graph.message import add_messages
from typing import Annotated, List, Dict, Any
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
import json
import os

from pydantic import BaseModel, Field
from typing import Literal
import uuid
from openai import OpenAI

# from xai_sdk import Client
# from xai_sdk.chat import user
# from xai_sdk.search import SearchParameters

## Setup credentials

In [24]:
load_dotenv()

True

## Setup and test basic llm call

In [ ]:
# Sample llm call - JUST A TEST
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash")
response = llm.invoke("What is the best crypto to buy at the currect moment on septemer 13th at 10:44 AM, 2025? research the market using websearch tools, use the following output format: [crypto_name]: [action (buy,sell,hold), [reasoning]]")
print(response.content)

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("XAI_API_KEY"),
    base_url="https://api.x.ai/v1",
)

I cannot fulfill this request as it asks for a prediction of future market conditions and specific financial advice.

As an AI, I do not have the ability to predict future events, especially market movements more than a year in advance (September 13th, 2025). The cryptocurrency market is highly volatile and influenced by a vast array of unpredictable factors such as technological developments, regulatory changes, macroeconomic shifts, global events, and market sentiment.

Furthermore, I am not a financial advisor, and providing specific 'buy,' 'sell,' or 'hold' recommendations constitutes financial advice, which I am not qualified or authorized to give. Such decisions depend heavily on an individual's financial situation, risk tolerance, and investment goals.

For informed investment decisions, it is crucial to:
*   **Conduct thorough personal research (DYOR).**
*   **Consult with a qualified financial advisor.**
*   **Understand the inherent risks associated with cryptocurrency invest

## Define input schema for kraken order api

In [ ]:

# Define Pydantic schema for Kraken order attributes
class KrakenOrderSchema(BaseModel):
    ordertype: Literal["limit", "market"] = Field(description="Type of order - limit or market")
    type: Literal["buy", "sell"] = Field(description="Buy or sell order")
    volume: str = Field(description="Order volume in base currency")
    pair: str = Field(description="Currency pair ID or ALTNAME (e.g., XBTUSD,ticker/currency)")
    price: str = Field(description="Price for limit orders")


## rough Example of user data

In [27]:
## Example of user data (rag from current portfolio)
current_holdings = """
- Bitcoin (BTC): 0.5 BTC
- Ethereum (ETH): 2.0 ETH  
- Solana (SOL): 50 SOL
- Cardano (ADA): 1000 ADA
"""

## Make llm call with grok search knowledge and sample wallet data

In [28]:

url = "https://api.x.ai/v1/chat/completions"
headers = {
"Content-Type": "application/json",
"Authorization": f"Bearer {os.getenv('XAI_API_KEY')}"
}
payload = {
"messages": [
{
"role": "user",
"content": """
# Cryptocurrency Portfolio Analysis and Investment Recommendation

## Current Portfolio
I currently hold the following cryptocurrency positions:


## Preferences:

## Research Request
Please conduct thorough market research on current cryptocurrency trends, market conditions, and emerging opportunities. Based on this research and my existing portfolio composition, provide a smart diversification recommendation on what i should buy.

## Analysis Requirements
1. **Market Research**: Analyze current market conditions, trends, and sentiment
2. **Portfolio Assessment**: Evaluate my current holdings and identify gaps or overexposure
3. **Diversification Strategy**: Recommend a action that would improve portfolio balance
4. **Risk Management**: Consider how the recommendation fits my risk profile

## Output Format
Token: [TOKEN_NAME]
Action: [BUY/SELL/HOLD],
Price: [PRICE],
Quantity: [QUANTITY],
Reasoning: [Detailed explanation including market analysis, portfolio fit, risk assessment, and diversification benefits]
"""
}
],
"search_parameters": {
"mode": "auto",
"max_search_results": 10,
"from_date": "2025-01-01",
"to_date": "2025-12-31"
},
"model": "grok-4"
}

response = requests.post(url, headers=headers, json=payload)
print(response.json())

#make another call to parse into structured output



{'id': '9c9e3a0b-65cf-ecf0-af18-b8d6d467ee61_us-east-1', 'object': 'chat.completion', 'created': 1757798889, 'model': 'grok-4-0709', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Token: BTC\nAction: BUY,\nPrice: $115,000,\nQuantity: 0.1,\nReasoning: Based on recent market research from sources like Exploding Topics and Forbes, the cryptocurrency market in 2025 is experiencing a strong bull run following volatility in Q1 due to US trade tariffs. As of September 13, 2025, the total market cap has risen to around $4.12 trillion, with Bitcoin (BTC) leading at approximately $115,000 per coin, up 18% YTD and outperforming traditional assets like gold and stocks. Key trends include AI integration in DeFi, growth in tokenized assets, and stablecoin-driven demand for altcoins, with sentiment on platforms like X showing optimism around BTC's role as a macro asset amid ETF inflows and treasury adoptions. Emerging opportunities highlight Solana (SOL) at $239 with ecosystem 

In [29]:
# Extract content from response
content = response.json()['choices'][0]['message']['content']
print(content)

Token: BTC
Action: BUY,
Price: $115,000,
Quantity: 0.1,
Reasoning: Based on recent market research from sources like Exploding Topics and Forbes, the cryptocurrency market in 2025 is experiencing a strong bull run following volatility in Q1 due to US trade tariffs. As of September 13, 2025, the total market cap has risen to around $4.12 trillion, with Bitcoin (BTC) leading at approximately $115,000 per coin, up 18% YTD and outperforming traditional assets like gold and stocks. Key trends include AI integration in DeFi, growth in tokenized assets, and stablecoin-driven demand for altcoins, with sentiment on platforms like X showing optimism around BTC's role as a macro asset amid ETF inflows and treasury adoptions. Emerging opportunities highlight Solana (SOL) at $239 with ecosystem expansion and memecoins like DOGE outperforming, but BTC remains the safest entry point for diversification.

Your current portfolio appears empty based on the provided details, which indicates a complete la

## Use recomendation returned from grok to create a structured output for the kraken api to fulfill and order

In [30]:
# Alternative implementation using Gemini and LangChain for structured output
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

# Initialize Gemini model
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

# Create output parser for structured data
parser = PydanticOutputParser(pydantic_object=KrakenOrderSchema)

# Create prompt template with format instructions
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert cryptocurrency analyst and portfolio manager. Your task is to parse and structure cryptocurrency trading recommendations from market analysis content.

    The input content follows this format:
    - Token: [TOKEN_NAME] (e.g., BTC (Bitcoin))
    - Action: [BUY/SELL/HOLD]
    - Reasoning: [Detailed market analysis, portfolio assessment, and risk evaluation]

    IMPORTANT: You must return a SINGLE JSON object (not an array) that matches the KrakenOrderSchema format. If multiple tokens are recommended, choose the PRIMARY recommendation only.
    
    {format_instructions}"""),
    ("user", """Parse the following cryptocurrency trading recommendation and structure it into the required JSON format:

    {content}

    Current Holdings: {current_holdings}

    Extract the PRIMARY token recommendation only. Return a single JSON object (not an array) with the token name, trading action, and detailed reasoning. Ensure all market analysis, portfolio considerations, and risk assessments are preserved in the structured output.""")
])

# Create the chain without parser first to see raw output
chain_raw = prompt | gemini_llm
raw_response = {}
gemini_completion = {}
try:
    # Execute the chain to get raw output first
    raw_response = chain_raw.invoke({
        "content": content,
        "current_holdings": current_holdings,
        "format_instructions": parser.get_format_instructions()
    })
    
    print("Raw Gemini response:")
    print(raw_response.content)
    print("\n" + "="*50 + "\n")
    
    # Try to parse the raw response
    try:
        # Clean the response content to extract JSON
        response_content = raw_response.content.strip()
        
        # If response starts with ```json, extract the JSON part
        if response_content.startswith("```json"):
            response_content = response_content.split("```json")[1].split("```")[0].strip()
        elif response_content.startswith("```"):
            response_content = response_content.split("```")[1].split("```")[0].strip()
        
        # Parse JSON
        parsed_json = json.loads(response_content)
        
        # If it's a list, take the first item
        if isinstance(parsed_json, list) and len(parsed_json) > 0:
            parsed_json = parsed_json[0]
            print("Warning: Received list, taking first item")
        
        # Validate with Pydantic
        gemini_completion = KrakenOrderSchema.model_validate(parsed_json)
        
        print("Successfully parsed Gemini + LangChain structured output:")
        print(gemini_completion)
        
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print("Raw content that failed to parse:")
        print(repr(response_content))
        
    except Exception as e:
        print(f"Pydantic validation error: {e}")
        print("Parsed JSON that failed validation:")
        print(parsed_json)

        
except Exception as e:
    print(f"Chain execution error: {e}")


Raw Gemini response:
```json
{
  "ordertype": "limit",
  "type": "buy",
  "volume": "0.1",
  "pair": "XBTUSD",
  "price": "115000"
}
```


Successfully parsed Gemini + LangChain structured output:
ordertype='limit' type='buy' volume='0.1' pair='XBTUSD' price='115000'


In [31]:
from pprint import pprint
print(gemini_completion)

ordertype='limit' type='buy' volume='0.1' pair='XBTUSD' price='115000'


## Create Web Browsing Tools


## Create LangGraph Web Browsing Agent


In [ ]:
# 

## Advanced LangGraph Implementation with State Management


In [33]:
# # Structured Output Example for Crypto Trading Data
# from openai import OpenAI
# from pydantic import BaseModel, Field
# from datetime import date
# from enum import Enum
# from typing import List, Optional

# # Pydantic Schemas for Crypto Trading Data

# class TradingAction(str, Enum):
#     BUY = "BUY"
#     SELL = "SELL"
#     HOLD = "HOLD"

# class RiskLevel(str, Enum):
#     LOW = "LOW"
#     MEDIUM = "MEDIUM"
#     HIGH = "HIGH"

# class MarketSignal(BaseModel):
#     signal_type: str = Field(description="Type of signal (technical, sentiment, fundamental)")
#     value: float = Field(description="Signal strength (-1 to 1)")
#     weight: float = Field(description="Weight of this signal in decision making")
#     source: str = Field(description="Source of the signal")

# class TradingRecommendation(BaseModel):
#     cryptocurrency: str = Field(description="Cryptocurrency symbol (e.g., BTC, ETH)")
#     action: TradingAction = Field(description="Recommended trading action")
#     confidence: float = Field(description="Confidence level (0-100)", ge=0, le=100)
#     reasoning: str = Field(description="Detailed explanation for the recommendation")
#     risk_level: RiskLevel = Field(description="Risk assessment of the trade")
#     position_size: float = Field(description="Recommended position size as percentage of portfolio", ge=0, le=100)
#     stop_loss: Optional[float] = Field(description="Stop loss price", default=None)
#     take_profit: Optional[float] = Field(description="Take profit price", default=None)
#     market_signals: List[MarketSignal] = Field(description="Supporting market signals")



# print("✅ Crypto trading schemas defined and client initialized")


In [34]:
# # Make structured output call for crypto trading recommendation
# completion = client.beta.chat.completions.parse(
#     model="grok-4",
#     messages=[
#         {
#             "role": "system", 
#             "content": """You are an expert cryptocurrency analyst. Analyze the given market data and provide structured trading recommendations. 
#             Consider technical analysis, market sentiment, fundamental factors, and risk assessment. 
#             Provide detailed reasoning and supporting signals for your recommendations."""
#         },
#         {
#             "role": "user", 
#             "content": """
#             Analyze the current cryptocurrency market and provide trading recommendations for Bitcoin and Ethereum.
            
#             Current Market Data:
#             - Bitcoin (BTC): $95,000 (+5.2% 24h, +15.8% 7d)
#             - Ethereum (ETH): $3,200 (+3.1% 24h, +12.4% 7d)
            
#             Market Context:
#             - Bull market conditions with strong institutional adoption
#             - Recent ETF approvals driving mainstream adoption
#             - DeFi TVL at all-time highs
#             - Positive regulatory developments in major markets
            
#             Technical Indicators:
#             - BTC: Above 50-day and 200-day moving averages, RSI at 65
#             - ETH: Strong support at $3,000, resistance at $3,500
            
#             News Sentiment:
#             - Positive: Major banks announcing crypto custody services
#             - Neutral: Regulatory clarity improving in key jurisdictions
#             - Risk: Potential market correction after recent gains
            
#             Please provide structured trading recommendations with detailed analysis.
#             """
#         }
#     ],
#     response_format=TradingRecommendation,
# )

# # Extract the parsed recommendation
# recommendation = completion.choices[0].message.parsed

# print("🎯 Structured Trading Recommendation:")
# print("=" * 50)
# print(f"Cryptocurrency: {recommendation.cryptocurrency}")
# print(f"Action: {recommendation.action}")
# print(f"Confidence: {recommendation.confidence}%")
# print(f"Risk Level: {recommendation.risk_level}")
# print(f"Position Size: {recommendation.position_size}%")
# print(f"Stop Loss: ${recommendation.stop_loss}" if recommendation.stop_loss else "Stop Loss: Not set")
# print(f"Take Profit: ${recommendation.take_profit}" if recommendation.take_profit else "Take Profit: Not set")
# print(f"\nReasoning: {recommendation.reasoning}")
# print(f"\nMarket Signals:")
# for signal in recommendation.market_signals:
#     print(f"  - {signal.signal_type}: {signal.value:.3f} (weight: {signal.weight}, source: {signal.source})")


In [35]:
# # Alternative: Multiple Recommendations Schema
# class PortfolioRecommendation(BaseModel):
#     recommendations: List[TradingRecommendation] = Field(description="List of trading recommendations")
#     portfolio_risk_score: float = Field(description="Overall portfolio risk score (0-100)", ge=0, le=100)
#     diversification_notes: str = Field(description="Notes on portfolio diversification")
#     market_outlook: str = Field(description="Overall market outlook and strategy")

# # Make call for multiple recommendations
# multi_completion = client.beta.chat.completions.parse(
#     model="grok-4",
#     messages=[
#         {
#             "role": "system", 
#             "content": """You are a portfolio manager specializing in cryptocurrency investments. 
#             Provide comprehensive portfolio recommendations considering diversification, risk management, and market conditions."""
#         },
#         {
#             "role": "user", 
#             "content": """
#             Create a diversified cryptocurrency portfolio recommendation based on current market conditions.
            
#             Available cryptocurrencies for analysis:
#             - Bitcoin (BTC): $95,000, Market Cap: $1.8T
#             - Ethereum (ETH): $3,200, Market Cap: $380B  
#             - Solana (SOL): $180, Market Cap: $80B
#             - Cardano (ADA): $0.45, Market Cap: $16B
#             - Chainlink (LINK): $12, Market Cap: $7B
            
#             Portfolio Requirements:
#             - Total portfolio value: $100,000
#             - Risk tolerance: Medium
#             - Investment horizon: 6-12 months
#             - Focus on diversification across different sectors (L1, DeFi, Infrastructure)
            
#             Provide recommendations for each cryptocurrency with position sizing and risk assessment.
#             """
#         }
#     ],
#     response_format=PortfolioRecommendation,
# )

# # Extract the portfolio recommendation
# portfolio = multi_completion.choices[0].message.parsed

# print("📊 Portfolio Recommendation:")
# print("=" * 60)
# print(f"Portfolio Risk Score: {portfolio.portfolio_risk_score}/100")
# print(f"Market Outlook: {portfolio.market_outlook}")
# print(f"Diversification Notes: {portfolio.diversification_notes}")
# print(f"\nIndividual Recommendations:")
# print("-" * 40)

# for i, rec in enumerate(portfolio.recommendations, 1):
#     print(f"\n{i}. {rec.cryptocurrency}")
#     print(f"   Action: {rec.action}")
#     print(f"   Position Size: {rec.position_size}% (${rec.position_size * 1000:.0f})")
#     print(f"   Confidence: {rec.confidence}%")
#     print(f"   Risk Level: {rec.risk_level}")
#     print(f"   Reasoning: {rec.reasoning[:100]}...")
